In [1]:
from utils import generate_array
from graphics import display_grid
from grid import Grid
import numpy as np

In [2]:
actions = {(0, 0):0, (-1, 0):1, (0, 1):2, (1, 0):3, (0, -1):4}

In [3]:
train_xs = []
train_ys = []

# makes 5x5 test grid and chooses random action until terminal state is reached
grid = Grid(5,random=True)
grid.agent_pos
#display_grid(grid)
#print(grid.current_state)
while not grid.terminal_state:
    print("")
    print(generate_array(grid))
    train_xs.append(generate_array(grid))
    action =  tuple(grid.legal_actions())[0]#random.choice(tuple(grid.legal_actions()))
    print(action)
    train_ys.append(actions[action])
    
    grid.T(action)
    #display_grid(grid)
    #print(grid.current_state)


[[3 0 0 0 0]
 [0 1 0 2 0]
 [0 0 0 4 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
(0, 0)

[[0 0 0 0 0]
 [3 1 0 2 0]
 [0 0 0 4 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
(0, 0)

[[0 0 0 0 0]
 [0 1 0 2 0]
 [3 0 0 4 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
(0, 0)

[[0 0 0 0 0]
 [0 1 0 2 0]
 [0 0 0 4 0]
 [3 0 0 0 0]
 [0 0 0 0 0]]
(0, 0)

[[0 0 0 0 0]
 [0 1 0 2 0]
 [0 0 0 4 0]
 [0 0 0 0 0]
 [3 0 0 0 0]]
(0, 0)


In [6]:
train_xs = np.array(train_xs, dtype=np.float32)
train_ys = np.array(train_ys, dtype=np.int64)
print(train_xs.shape, train_ys.shape)

(5, 5, 5) (5,)


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 5, padding=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(10, 30, 3, padding=1)
        self.fc1 = nn.Linear(30 * 1 * 1, 5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 30 * 1 * 1)
        x = self.fc1(x)
        return x
    
net = Net()

In [8]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# input shape b x c x h x w to net
# so need to unsqueeze to make channel dimension of 1, could also make each agent it's own channel instead
train_xs = torch.from_numpy(train_xs).unsqueeze(1)
train_ys = torch.from_numpy(train_ys)

In [9]:

for epoch in range(300):  # loop over the dataset multiple times

    running_loss = 0.0
    
    inputs, labels = train_xs, train_ys

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # print statistics
    running_loss += loss.item()
    if epoch%10==9:
        print('Epoch:{}, loss: {:.3f}'.format(epoch + 1, running_loss))
        running_loss = 0.0

Epoch:10, loss: 1.443
Epoch:20, loss: 1.098
Epoch:30, loss: 0.709
Epoch:40, loss: 0.361
Epoch:50, loss: 0.155
Epoch:60, loss: 0.072
Epoch:70, loss: 0.041
Epoch:80, loss: 0.028
Epoch:90, loss: 0.021
Epoch:100, loss: 0.017
Epoch:110, loss: 0.014
Epoch:120, loss: 0.012
Epoch:130, loss: 0.011
Epoch:140, loss: 0.010
Epoch:150, loss: 0.009
Epoch:160, loss: 0.008
Epoch:170, loss: 0.007
Epoch:180, loss: 0.007
Epoch:190, loss: 0.006
Epoch:200, loss: 0.006
Epoch:210, loss: 0.005
Epoch:220, loss: 0.005
Epoch:230, loss: 0.005
Epoch:240, loss: 0.005
Epoch:250, loss: 0.004
Epoch:260, loss: 0.004
Epoch:270, loss: 0.004
Epoch:280, loss: 0.004
Epoch:290, loss: 0.004
Epoch:300, loss: 0.003
